To run this script you need a different environment, which includes xarray and pyproj. You can get the data as indicated below via opendata.dwd.de

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
from tqdm import tqdm
import pyproj

In [ ]:

#read list of locations
Well_list = pd.read_csv("./locations.csv",
                        sep=';',header = 0,encoding = 'Latin1')

# transform lat lon in hyras projection: EPSG:3034
proj_wgs84 = pyproj.Proj(init="epsg:4326")
proj_hyras = pyproj.Proj(init="epsg:3034") # ETRS89 / LCC Europe (EPSG:3034)
Well_list['XHYRAS'], Well_list['YHYRAS'] = pyproj.transform(proj_wgs84, proj_hyras, Well_list['Lon'], Well_list['Lat'])




In [ ]:

#get the data here: https://opendata.dwd.de/climate_environment/CDC/grids_germany/daily/hyras_de/

#read nc files
file = './pr_hyras_1_1931_2020_v5-0_de.nc'
ds_P = xr.open_dataset(file)

file = './tas_hyras_5_1951_2020_v5-0_de.nc'
ds_T = xr.open_dataset(file)



In [ ]:
#loop locations
for loc in tqdm(range(Well_list.shape[0])):
    
    ID = Well_list['ID'][loc]
    yhyras = Well_list['YHYRAS'][loc]
    xhyras = Well_list['XHYRAS'][loc]

    #Precipitation
    data_P = np.asarray(ds_P.pr.sel(x=xhyras, y=yhyras, method='nearest'),dtype='float64')
    t_P = ds_P.time.values
    t_P = pd.to_datetime(t_P)
    
    P_daily = pd.DataFrame(data=data_P,index=t_P,columns=((['P'])))
    
    P_weekly = P_daily.resample("W-Mon").sum()
    P_weekly['P'] = np.round(P_weekly,2)
    P_weekly['P'] = P_weekly['P'].astype('float64')

    #Temperature
    data_T = np.asarray(ds_T.tas.sel(x=xhyras, y=yhyras, method='nearest'),dtype='float64')
    t_T = ds_T.time.values
    t_T = pd.to_datetime(t_T)
    
    T_daily = pd.DataFrame(data=data_T,index=t_T,columns=((['T'])))
            
    T_weekly = T_daily.resample("W-Mon").mean()
    T_weekly['T'] = np.round(T_weekly,2)
    T_weekly['T'] = T_weekly['T'].astype('float64')
    
    #join P and T
    weekly = pd.concat([P_weekly,T_weekly],axis = 1,join = 'inner',ignore_index=False, copy = True)
    
    
    # save 
    path = '/.'# change accordingly
    file = path+ID+'_Hyras_weekly.csv'
    weekly.to_csv(file,sep=',',index_label = 'Date')

    